# IMPORTING LIBRARIES

In [ ]:
#Importing Libraries
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix 
from imblearn.combine import SMOTETomek
import string
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

nltk.download('stopwords')


stop_words = set(stopwords.words("english"))


#havent used yet

default_stopwords = stopwords.words('english')
default_tokenizer=RegexpTokenizer(r"\w+")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **Dataset**

In [ ]:
#Reading Dataset
data = pd.read_csv("/content/fake_job_postings.csv")
data.head()
data.describe()
data = data.replace(np.nan, '', regex=True)
data.columns
data.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

## **DATA PREPROCESSING**

In [ ]:
data['text'] = data[['title', 'department','company_profile','description'
,'requirements','benefits']].apply(lambda x: ' '.join(x), axis = 1)

In [ ]:
data.drop(['job_id', 'location','title','salary_range' ,'department','salary_range','company_profile',
           'description','requirements','benefits'], axis=1, inplace=True)

In [ ]:
data.columns
data_columns = data.columns.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_columns = ['telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function']
lb_make = LabelEncoder()
for i in label_columns:
  data[i] = lb_make.fit_transform(data[i])


In [ ]:
data_columns = data_columns[-1:] + data_columns[:-1]
data = data[data_columns]
data.head()

,text,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,"Marketing Intern Marketing We're Food52, and w...",0,1,0,3,5,0,0,23,0
1,Customer Service - Cloud Video Production Succ...,0,1,0,2,7,0,76,8,0
2,Commissioning Machinery Assistant (CMA) Valor...,0,1,0,0,0,0,0,0,0
3,Account Executive - Washington DC Sales Our pa...,0,1,0,2,6,2,23,32,0
4,Bill Review Manager SpotSource Solutions LLC ...,0,1,1,2,6,2,52,17,0


In [ ]:
# we are removing the words from the stop words list
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
## Defining the utility functions

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',str(text))


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', str(text))


def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(text))


def remove_punctuation(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)


    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def final_preprocess(text):
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\n', ' ')
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = ' '.join(e for e in text.split() if e.lower() not in stopwords)
    text = text.lower()
    ps = PorterStemmer()
    text = ps.stem(text)
    return text

In [ ]:
#Cleaning USING Utility Functions described above

data['text'] = remove_URL(str(data['text']))
data['text'] = remove_emoji(str(data['text']))
data['text'] = remove_html(str(data['text']))
data['text'] = remove_punctuation(str(data['text']))
data['text'] = final_preprocess(str(data['text']))

In [ ]:
data.head()

,text,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,0 0 0 0 0 marketing 1 0 0 0 0 marketing 2 0 0 ...,0,1,0,3,5,0,0,23,0
1,0 0 0 0 0 marketing 1 0 0 0 0 marketing 2 0 0 ...,0,1,0,2,7,0,76,8,0
2,0 0 0 0 0 marketing 1 0 0 0 0 marketing 2 0 0 ...,0,1,0,0,0,0,0,0,0
3,0 0 0 0 0 marketing 1 0 0 0 0 marketing 2 0 0 ...,0,1,0,2,6,2,23,32,0
4,0 0 0 0 0 marketing 1 0 0 0 0 marketing 2 0 0 ...,0,1,1,2,6,2,52,17,0


In [ ]:
from google.colab import drive
drive.mount("/content/drive")
f = open('/content/drive/My Drive/Colab Notebooks/glove.840B.300d.txt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Making a dictionary of the words and their vector representation

embeddings_index = {}
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vectors representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

In [ ]:

'''
Below is a uliity function that takes sentenes as a input and return the vector representation of the same
Method adopted is similar to average word2vec.

i am summing up all the vector representation of the words from the glove and 
then taking the average by dividing with the number of words involved
'''
glove_words =  set(embeddings_index.keys())
def convert_sen_to_vec(sentence):
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence
    for word in sentence.split():
        if word in glove_words:
            vector += embeddings_index[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    return vector

In [ ]:
converted_data = []

for i in range(0, data.shape[0]):
    converted_data.append(convert_sen_to_vec(data['text'][i]))
_1 = pd.DataFrame(converted_data)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data[['required_education', 
      'required_experience',
      'employment_type']] = StandardScaler().fit_transform(data[['required_education',
                                                                 'required_experience',
                                                                 'employment_type']])

In [ ]:
data.drop(["text"], axis=1, inplace=True)
main_data = pd.concat([_1,data], axis=1)

main_data.head()

X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

In [ ]:

from sklearn.model_selection import train_test_split

smk  = SMOTETomek(random_state=42)
X_res,Y_res= smk.fit_resample(X,Y)

X_train, X_test, Y_train, Y_test = train_test_split(X_res, Y_res ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

## **TRAINING MODELS**-

### **XGBOOST MODEL**

In [ ]:
xgb_model = xgb.XGBClassifier(learning_rate=0.02, n_estimators=300)

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,10,20,40]
        }

clf = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=10, scoring='f1', cv=4)
clf.fit(X_train, Y_train)

RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(learning_rate=0.02,
                                           n_estimators=300),
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5, 10, 20, 40],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   scoring='f1')

In [ ]:
print("Average of the best f1-score in various folds during cross validation = ",clf.best_score_)
print("The best parameters found during k-fold cross validation is = ",clf.best_params_)

Average of the best f1-score in various folds during cross validation =  0.924513649087151
The best parameters found during k-fold cross validation is =  {'subsample': 0.8, 'min_child_weight': 10, 'max_depth': 10, 'gamma': 1, 'colsample_bytree': 0.8}


## **BIDIRECTIONAL LSTM**

In [ ]:
embedding_vector_features=50
lstm_model=Sequential()
lstm_model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
lstm_model.add(Bidirectional(LSTM(100))) #Birecectional Input
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(1,activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(lstm_model.summary())

In [ ]:
lstm_model = lstm.XGBClassifier(learning_rate=0.02, n_estimators=300)

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,10,20,40]
        }

clf = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=10, scoring='f1', cv=4)
clf.fit(X_train, Y_train)

In [ ]:
Y_prediction = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_prediction))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4284
           1       0.94      0.92      0.93      4214

    accuracy                           0.93      8498
   macro avg       0.93      0.93      0.93      8498
weighted avg       0.93      0.93      0.93      8498

